In [2]:
import pandas as pd

In [ ]:
HOUR = 3600
DAY = HOUR * 24

timeframes = {
    '-12h': HOUR * 8,
    '12h': HOUR * 12,
    '1d': DAY,
    '2d': DAY * 2,
    '7d': DAY * 7,
    '14d': DAY * 14,
}

In [4]:
data = pd.read_parquet('../../data/bitcoin_twitter_raw/part_0.parquet')
data = data.iloc[:10000][['created_at']]
data.sort_values(by=['created_at'], ascending=True, inplace=True)